In [7]:
import warnings
warnings.filterwarnings("ignore")
import torch
import torchvision.transforms as transforms
import numpy as np
import glob
import matplotlib.pyplot as plt
from PIL import Image

In [8]:
import sys
sys.path.append("../pyfiles/")
from util import MinMax, image_from_output
from util_assignment import transform

# Introduction
---
課題は、データセットの実装とモデルの構造定義から構成されています。
その中でも前者の課題をこのノートブックで行います。
空欄を埋めて提出してください。
授業では使っていない[fashion_dataset2](https://drive.google.com/drive/folders/1YmKP_K9b9ZsZNWonA0t9FuDp6OSH16yW?usp=sharing)を用いるので、`00_build_dataset2.ipynb`を使って作成しておいてください．

---
Your assignment includes dataset implementation and model structuring. This notebook contains a form of the former homework, so please submit this notebook for evaluation. It requires [fashion_dataset2](https://drive.google.com/drive/folders/1YmKP_K9b9ZsZNWonA0t9FuDp6OSH16yW?usp=sharing), so please visit `00_build_dataset2.ipynb` to build it.

---

# Fashion_dataset2
---
データセットの構成はほとんどfashion_dataset1と同じです。
異なるのは、クラスのフォルダの名前だけです。
クラス番号の代わりに、クラスのラベル名になっています。

---
The major features are almost the same except for the name of folder. Class labels are employed as the name instead of class id (number). The tree structure is:
```
fashion_dataset2
├── T-shirt 
│   ├── 0_0001.png
│   ├── 0_0002.png
│   └── ...
├── Trouser
│   ├── 0_0001.png
│   ├── 0_0002.png
│   └── ...
├── Dress
│   ├── 0_0001.png
│   ├── 0_0002.png
│   └── ...
├── ...
```

---

# Homework 1: Dataset Modification
---
この課題では、履物のクラス（sandals, sneakers, and ankle boots）だけを抽出するために、データセット関数の変更をすることです。
このセルから下の２つのセルが変更対象で、変更する部分は強調してあります。
`01_classification.ipynb`を参考にして頑張ってください。

**ヒント: "class_labels" が引数に追加されています。**

---
In the first assignment, you are tasked with modifying dataset function to extract data that belong to footwear classes, which include sandals, sneakers, and ankle boots. The cells you need to modify are just two and I underscored the part you should change or add. Please refer to the lesson notebook `01_classification.ipynb` for some clues.

**Hint: "class_labels" is added to arguments.**

---

In [9]:
class Dataset_Fashion_MNIST(torch.utils.data.Dataset):
    def __init__(self, root, classes, class_labels, mode="train", transform=None, balance=[0.7,0.15,0.15], each_data_num=10000000):
        # Hint: "class_labels" is added to arguments.
        
        self.transform = transform
        self.images = []
        self.labels = []

        images = {} 
        labels = {}
        
        for cl in classes:
            
            # -------------- add the code below ------------------- #
            # ----------------------------------------------------- #
            path_list.sort()
            path_list = path_list[:each_data_num]
            
            train_num = int(balance[0]*len(path_list))
            val_num = int(balance[1]*len(path_list))
            test_num = int(balance[2]*len(path_list))
            
            if mode=="train":
                path_list = path_list[:train_num]
            elif mode=="val":
                path_list = path_list[train_num:train_num+val_num]
            elif mode=="test":
                path_list = path_list[-test_num:]
                
            images[str(cl)] = path_list
            labels[str(cl)] = [cl]*len(path_list)
            
        # combine them together
        for label in classes:
            for image, label in zip(images[str(label)], labels[str(label)]):
                self.images.append(image)
                self.labels.append(label)

    def __getitem__(self, index):
        
        image = self.images[index]
        label = self.labels[index]
        
        with open(image, 'rb') as f:
            image = Image.open(f)
            image = image.convert("L")
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label
    
    def __len__(self):
        return len(self.images)

In [18]:
# -------------- modify the code below ------------------- #
class_labels = ["T-shirt", "Trouser", "Dress", "Coat", "Sandal", "Sneaker", "Bag", "Ankle_boot"]
# -------------------------------------------------------- #

root = "../dataset/fashion-MNIST/fashion_dataset2/"
classes = range(len(class_labels))

---
以下のコードを実行して、実装のチェックをしてみましょう。出力結果が以下のようになれば成功です。

---
Run the code below to check your implementation. The output should be:

<img src="./../data/images/answer1.png" width="300">

---

In [ ]:
dataset = Dataset_Fashion_MNIST(root, classes, class_labels, "test", transform["test"], each_data_num=2000)
display_num = 4
each_num = int(len(dataset)/len(classes))
fig = plt.figure(figsize=(3.75*len(classes), display_num*4))
for i in range(display_num):
    for cl in classes:
        ax = fig.add_subplot(display_num, len(classes), len(classes)*i+cl+1)
        index = i + cl*each_num
        x, label = dataset[index]
        ax.imshow(image_from_output(x)[0])
        ax.set_title(f"{class_labels[cl]}", fontsize=20)